In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

## Load Model

In [2]:
import sys
sys.dont_write_bytecode = True
from model import IntrusionDetector

model = torch.load('save/intrusion_detector_full.pth', weights_only=False)
model.eval()

IntrusionDetector(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('resources/processed_cybersecurity_intrusion_data.csv')

X = df.drop('attack_detected', axis=1)
y = df['attack_detected']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
# X_test = X_test.astype(np.float32)

pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-1.403600,-0.016346,-0.253324,0.0,-1.467959,-0.419989,0.663147,-0.580862,-1.080509,2.343538,-0.505989,-0.231472,-0.235715
1,-1.519545,-1.544690,-0.427510,0.0,-0.500779,-0.419989,0.663147,-0.580862,0.925490,-0.426705,-0.505989,-0.231472,-0.235715
2,-0.859160,2.530894,-0.536853,0.0,0.466400,-0.419989,0.663147,-0.580862,-1.080509,2.343538,-0.505989,-0.231472,-0.235715
3,0.915311,-0.525794,-0.836911,0.0,-0.500779,-0.419989,-1.507960,1.721581,0.925490,-0.426705,-0.505989,-0.231472,-0.235715
4,0.476735,-0.525794,0.245078,0.0,0.466400,-0.419989,0.663147,-0.580862,-1.080509,2.343538,-0.505989,-0.231472,-0.235715
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,0.138980,1.002550,-0.647468,0.0,-1.467959,-0.419989,0.663147,-0.580862,0.925490,-0.426705,-0.505989,-0.231472,-0.235715
1904,-1.454011,-0.525794,-0.666539,0.0,1.433580,-0.419989,0.663147,-0.580862,0.925490,-0.426705,-0.505989,-0.231472,-0.235715
1905,-0.183651,-0.016346,-0.255867,0.0,-0.500779,-0.419989,-1.507960,1.721581,0.925490,-0.426705,-0.505989,-0.231472,-0.235715
1906,-0.339925,-1.544690,-0.972954,0.0,-0.500779,2.381015,0.663147,-0.580862,0.925490,-0.426705,-0.505989,-0.231472,-0.235715


## Wrap model with ART

In [4]:
from art.estimators.classification import PyTorchClassifier

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(X.shape[1],),
    nb_classes=2,
    clip_values=(X_test.min(), X_test.max())
)

/home/joe/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Evaluate Robustness

In [42]:
from art.attacks.evasion import FastGradientMethod

attack = FastGradientMethod(estimator=classifier, eps=0.5)
X_test_adv = attack.generate(x=(X_test.astype(np.float32)))

predictions = classifier.predict(X_test_adv)

y_pred = (torch.sigmoid(torch.from_numpy(predictions)).numpy() > 0.5).astype(int)

accuracy = np.mean(y_pred.flatten() == y_test)
print("Adversarial accuracy:", accuracy)

Adversarial accuracy: 0.7117400419287212


In [10]:
type(y_test)

pandas.core.series.Series

In [17]:
X_test.shape

(1908, 13)

In [18]:
y_test.shape

(1908,)

In [22]:
(y_test.values.reshape(-1, 1)).shape

(1908, 1)

In [32]:
torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).long()

tensor([[0],
        [0],
        [1],
        ...,
        [0],
        [0],
        [0]])

## Defending the Model

In [37]:
from art.defences.trainer import AdversarialTrainer

adversarial_classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(X.shape[1],),
    nb_classes=2,
    clip_values=(X_test.min(), X_test.max())
)

# y_test_tmp = y_test.to_numpy(np.float32).reshape(-1, 1)
y_test_tmp = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

trainer = AdversarialTrainer(adversarial_classifier, attacks=attack)
trainer.fit(X_test, y_test_tmp)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Target size (torch.Size([32])) must be the same as input size (torch.Size([32, 1]))